<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [2]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# URL of the Falcon 9 and Falcon Heavy Launches Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform HTTP GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("HTML content fetched successfully")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

# Optional: Displaying a snippet of the HTML content
print(html_content[:1000])


HTML content fetched successfully
b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-c

Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup

# Assuming html_content contains the HTML response from the previous step
# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Optional: Print the title of the page to verify the BeautifulSoup object
print(soup.title.string)


List of Falcon 9 and Falcon Heavy launches - Wikipedia


from bs4 import BeautifulSoup

# Assuming html_content contains the HTML response from the previous step
# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Optional: Print the title of the page to verify the BeautifulSoup object
print(soup.title.string)
Print the page title to verify if the `BeautifulSoup` object was created properly 


In [8]:
# Use soup.title attribute
import requests
from bs4 import BeautifulSoup

# URL of the Falcon 9 and Falcon Heavy Launches Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform HTTP GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("HTML content fetched successfully")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Print the title of the page to verify the BeautifulSoup object
print("Page Title:", soup.title.string)


HTML content fetched successfully
Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [9]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
import requests
from bs4 import BeautifulSoup

# URL of the Falcon 9 and Falcon Heavy Launches Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform HTTP GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("HTML content fetched successfully")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Locate the first table in the HTML content
table = soup.find_all('table')[0]

# Extract column names from the table header
headers = [header.text.strip() for header in table.find_all('th')]

# Print the column names
print("Column Names:", headers)


HTML content fetched successfully
Column Names: ['Flight No.', 'Date andtime (UTC)', 'Version,booster[c]', 'Launchsite', 'Payload[d]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '195', '196', 'FH 5', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', 'FH 6', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', 'FH 7', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', 'FH 8', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', 'FH 9', '285']


Starting from the third table is our target table contains the actual launch records.


In [10]:
# Let's print the third table and check its content
# first_launch_table = html_tables[2]
# print(first_launch_table)
import requests
from bs4 import BeautifulSoup

# URL of the Falcon 9 and Falcon Heavy Launches Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform HTTP GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("HTML content fetched successfully")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Locate all tables in the HTML content
html_tables = soup.find_all('table')

# Ensure there are at least three tables in the HTML content
if len(html_tables) >= 3:
    # Select the third table (index 2)
    first_launch_table = html_tables[2]

    # Print the content of the third table
    print(first_launch_table)
else:
    print("There are fewer than three tables on the page.")


HTML content fetched successfully
<table class="wikitable" style="width:100%; font-size:1.00em; line-height:1.5em;">
<tbody><tr class="is-sticky">
<th scope="col" style="width: 10%;">Date and time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>booster</a><sup class="reference" id="cite_ref-booster_27-2"><a href="#cite_note-booster-27">[c]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_28-2"><a href="#cite_note-Dragon-28">[d]</a></sup>
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th></tr>
<tr>
<td rowspan="2">23 June 2024<br/>17:15<sup class="reference" id="cite_ref-nextSFupcoming_369-1"><a href="#cite_note-nextSFupcoming-369">[361]</a></sup>
</td>
<td><a href="/wiki/Falcon_9_Block_5" title="Falcon 9 Block 5">F9 B5</a> ♺<br/><a hr

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [11]:
column_names = []
import requests
from bs4 import BeautifulSoup

# Provided extract_column_from_header function for demonstration
def extract_column_from_header(th):
    return th.text.strip()

# URL of the Falcon 9 and Falcon Heavy Launches Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform HTTP GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("HTML content fetched successfully")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Locate all tables in the HTML content
html_tables = soup.find_all('table')

# Ensure there are at least three tables in the HTML content
if len(html_tables) >= 3:
    # Select the third table (index 2)
    first_launch_table = html_tables[2]
    
    # Extract column names from the table header
    column_names = []
    for th in first_launch_table.find_all('th'):
        name = extract_column_from_header(th)
        if name is not None and len(name) > 0:
            column_names.append(name)
    
    # Print the column names
    print("Column Names:", column_names)
else:
    print("There are fewer than three tables on the page.")

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


HTML content fetched successfully
Column Names: ['Date and time (UTC)', 'Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer']


Check the extracted column names


In [12]:
print(column_names)

['Date and time (UTC)', 'Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Provided extract_column_from_header function for demonstration
def extract_column_from_header(th):
    return th.text.strip()

# URL of the Falcon 9 and Falcon Heavy Launches Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform HTTP GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("HTML content fetched successfully")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Locate all tables in the HTML content
html_tables = soup.find_all('table')

# Ensure there are at least three tables in the HTML content
if len(html_tables) >= 3:
    # Select the third table (index 2)
    first_launch_table = html_tables[2]
    
    # Extract column names from the table header
    column_names = []
    for th in first_launch_table.find_all('th'):
        name = extract_column_from_header(th)
        if name is not None and len(name) > 0:
            column_names.append(name)
    
    # Print the column names
    print("Column Names:", column_names)

    # Create and initialize launch_dict
    launch_dict = dict.fromkeys(column_names)
    
    # Remove irrelevant column
    if 'Date and time (UTC)' in launch_dict:
        del launch_dict['Date and time (UTC)']
    
    # Initialize each key with an empty list
    for key in launch_dict:
        launch_dict[key] = []

    # Add new columns and initialize them
    launch_dict['Version Booster'] = []
    launch_dict['Booster landing'] = []
    launch_dict['Date'] = []
    launch_dict['Time'] = []
    
    # Print the initialized dictionary keys to verify
    print("Initialized launch_dict keys:", launch_dict.keys())
else:
    print("There are fewer than three tables on the page.")


HTML content fetched successfully
Column Names: ['Date and time (UTC)', 'Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer']
Initialized launch_dict keys: dict_keys(['Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer', 'Version Booster', 'Booster landing', 'Date', 'Time'])


Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Placeholder functions for processing the table data
def extract_column_from_header(th):
    return th.text.strip()

def date_time(cell):
    date_time_str = cell.text.strip().split(' ')
    return date_time_str if len(date_time_str) == 2 else [date_time_str[0], ""]

def booster_version(cell):
    return cell.text.strip()

def get_mass(cell):
    mass_text = cell.text.strip().split(' ')[0]
    return mass_text if mass_text else "0"

def landing_status(cell):
    return cell.text.strip()

# URL of the Falcon 9 and Falcon Heavy Launches Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches"

# Perform HTTP GET request to fetch the HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.content
    print("HTML content fetched successfully")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

# Create a BeautifulSoup object from the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Locate all tables in the HTML content
html_tables = soup.find_all('table')

# Ensure there are at least three tables in the HTML content
if len(html_tables) >= 3:
    # Select the third table (index 2)
    first_launch_table = html_tables[2]
    
    # Extract column names from the table header
    column_names = []
    for th in first_launch_table.find_all('th'):
        name = extract_column_from_header(th)
        if name is not None and len(name) > 0:
            column_names.append(name)
    
    # Print the column names
    print("Column Names:", column_names)

    # Create and initialize launch_dict
    launch_dict = dict.fromkeys(column_names)
    
    # Remove irrelevant column
    if 'Date and time (UTC)' in launch_dict:
        del launch_dict['Date and time (UTC)']
    
    # Initialize each key with an empty list
    for key in launch_dict:
        launch_dict[key] = []

    # Add new columns and initialize them
    launch_dict['Version Booster'] = []
    launch_dict['Booster landing'] = []
    launch_dict['Date'] = []
    launch_dict['Time'] = []
    
    # Print the initialized dictionary keys to verify
    print("Initialized launch_dict keys:", launch_dict.keys())
    
    # Extract and fill the launch records
    extracted_row = 0
    for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
        for rows in table.find_all("tr"):
            if rows.th and rows.th.string and rows.th.string.strip().isdigit():
                flight_number = rows.th.string.strip()
                row = rows.find_all('td')
                extracted_row += 1
                
                # Flight Number
                launch_dict['Flight No.'].append(flight_number)
                
                # Date and Time
                datatimelist = date_time(row[0])
                date = datatimelist[0].strip(',')
                time = datatimelist[1]
                launch_dict['Date'].append(date)
                launch_dict['Time'].append(time)
                
                # Booster Version
                bv = booster_version(row[1])
                if not bv:
                    bv = row[1].a.string
                launch_dict['Version Booster'].append(bv)
                
                # Launch Site
                launch_site = row[2].a.string
                launch_dict['Launch site'].append(launch_site)
                
                # Payload
                payload = row[3].a.string
                launch_dict['Payload'].append(payload)
                
                # Payload Mass
                payload_mass = get_mass(row[4])
                launch_dict['Payload mass'].append(payload_mass)
                
                # Orbit
                orbit = row[5].a.string
                launch_dict['Orbit'].append(orbit)
                
                # Customer
                customer = row[6].a.string
                launch_dict['Customer'].append(customer)
                
                # Launch Outcome
                launch_outcome = list(row[7].strings)[0]
                launch_dict['Launch outcome'].append(launch_outcome)
                
                # Booster Landing
                booster_landing = landing_status(row[8])
                launch_dict['Booster landing'].append(booster_landing)
    
    # Convert launch_dict to a DataFrame
    df = pd.DataFrame(launch_dict)
    print(df.head())
else:
    print("There are fewer than three tables on the page.")


HTML content fetched successfully
Column Names: ['Date and time (UTC)', 'Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer']
Initialized launch_dict keys: dict_keys(['Version,booster[c]', 'Launch site', 'Payload[d]', 'Orbit', 'Customer', 'Version Booster', 'Booster landing', 'Date', 'Time'])


KeyError: 'Flight No.'

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
